In [1]:
import pandas as pd

%matplotlib notebook

In [2]:
section = pd.read_csv('../preprocesses/section_data/result_section.csv')

section = section[['SectionID', 'FromNodeID', 'ToNodeID']].copy()

section['SectionID'] = section['SectionID'].astype(str)
section['FromNodeID'] = section['FromNodeID'].astype(str)
section['ToNodeID'] = section['ToNodeID'].astype(str)

#section = section[406:411]

with pd.option_context('display.max_rows', None, 'display.max_columns', 6):
    print(section)
    
### What am I doing? ###
# 
# open section: contains connections between nodes

                              SectionID FromNodeID  \
0                              42571222   42901266   
1                              42533575   42890834   
2                              43939209   42787380   
3                              43825506   42698357   
4                              43971553   42703621   
5                              94588448   52857530   
6                     42783662-SPOTLOAD   42783662   
7                              43895417   52889602   
8                              42533570   42890835   
9                              43906026   42785340   
10                             43939204   42660293   
11                    42787380-SPOTLOAD   42787380   
12                             93849829   93848443   
13                             43884803   42686073   
14                             42571224   42901268   
15                    42769314-SPOTLOAD   42769314   
16                             52107522   42890831   
17                          

In [3]:
intermediate_nodes = pd.read_csv('../preprocesses/section_data/result_intermediate_nodes.csv')


intermediate_nodes = intermediate_nodes[['SectionID','SeqNumber', 'CoordX', 'CoordY']].copy()

# intermediate_nodes = intermediate_nodes.rename(index=str, columns={"SectionID": "NodeID"})

del_coord = intermediate_nodes[intermediate_nodes["CoordX"] >= 555000]

#del_coord = del_coordX[del_coordX["CoordY"] >= 150000]

del_coord = pd.DataFrame(del_coord["SectionID"].reset_index(drop=True))
del_coord["SectionID"] = del_coord["SectionID"].astype(str)


#with pd.option_context('display.max_rows', None, 'display.max_columns', 6):
#    print(del_coord)
    
intermediate_nodes = intermediate_nodes[~intermediate_nodes['SectionID'].isin(del_coord['SectionID'])]


intermediate_nodes["SectionID"] = intermediate_nodes["SectionID"].astype(str)
intermediate_nodes=intermediate_nodes.reset_index(drop=True)

#intermediate_nodes = intermediate_nodes[0:20]
with pd.option_context('display.max_rows', None, 'display.max_columns', 6):
    print(intermediate_nodes)

### What am I doing? ###
#
# open intermediate nodes: contain nodes which are between nodes
# intermediate nodes which are out of the considered intervall are deleted

      SectionID  SeqNumber         CoordX         CoordY
0     160232402          0  554220.352000  146210.115000
1      43966944          0  554328.299000  146566.836000
2      43966944          1  554335.999000  146566.836000
3      43966944          2  554336.356000  146574.455000
4      43966944          3  554331.866000  146575.979000
5      43966944          4  554324.583000  146577.692000
6      43966944          5  554319.233000  146578.554000
7      43966944          6  554313.943000  146579.139000
8      43966944          7  554306.375000  146579.139000
9      43966944          8  554299.766000  146578.963000
10     43966944          9  554296.332000  146578.641000
11     43966944         10  554291.171000  146577.829000
12     43966944         11  554283.304000  146576.762000
13     43966944         12  554279.030000  146576.302000
14     43968073          0  554230.035000  145709.713000
15     43968073          1  554231.949000  145718.190000
16     43968073          2  554

In [5]:
# Algorithm to create intermediate_nodes.csv

section_complet = pd.DataFrame()


section_dict = {}

for i, row in section.iterrows(): 
    #for j, row in intermediate_nodes.iterrows():
    j = 0
    while j < 3440: # find command for this
        # start node
        #j = k
        if section['SectionID'][i] == intermediate_nodes['SectionID'][j]:

            section_dict.update({'SectionID': section['SectionID'][i],
                                 'FromNodeID': section['FromNodeID'][i],
                                 'ToNodeID': intermediate_nodes['SectionID'][j] +str('(') + str('0') +str(')') })
            section_complet = section_complet.append(section_dict.copy(), ignore_index=True)
            
            for m in range(1,200):
                if j + m >=3440:
                    break
                    j = 3440
                else:
                # interconnection nodes j+m
                    if intermediate_nodes['SectionID'][j] == intermediate_nodes['SectionID'][j+m]:

                        section_dict.update({'SectionID': section['SectionID'][i],
                                             'FromNodeID': intermediate_nodes['SectionID'][j]+str('(') +str(m-1) +str(')'), 
                                             'ToNodeID':  intermediate_nodes['SectionID'][j]+str('(') +str(m) +str(')') })
                        section_complet = section_complet.append(section_dict.copy(),ignore_index=True)
                        continue 
                        

                    else:

                        section_dict.update({'SectionID': section['SectionID'][i],
                                             'FromNodeID': intermediate_nodes['SectionID'][j]+str('(') +str(m-1) +str(')'),
                                             'ToNodeID':  section['ToNodeID'][i] })
                        section_complet = section_complet.append(section_dict.copy(),ignore_index=True)
                        #k = j + m +1
                        break
                        j = 3440
                        
                    

        else:
            j = j+1


section_complet = pd.DataFrame(section_complet)


with pd.option_context('display.max_rows', None, 'display.max_columns', 6):
    print(section_complet)
section_complet.to_csv(r'../preprocesses/section_data/intermediate_nodes_while.csv', header=True, index=True, mode='a')

### What am I doing? ###
#
# Rearrange the section file, by adding the intermediate nodes with (j) based on their SeqNumber
# Two solutions: For loop: Final list still have the direct connection between the "original" nodes. Need to be deleted afterwards
#                While loop: takes longer, but file is "clean"

KeyboardInterrupt: 

In [ ]:
#intermediate_nodes_changed = pd.read_csv('../preprocesses/section_data/intermediate_nodes.csv')

#intermediate_nodes_changed = intermediate_nodes_changed[['SectionID', 'FromNodeID', 'ToNodeID']].copy()

#intermediate_nodes_changed

In [ ]:
#for j,row in intermediate_nodes.iterrows():
#    intermediate_nodes['SectionID'][j] = str(intermediate_nodes['SectionID'][j]) + str('(') + str(intermediate_nodes['SeqNumber'][j]) +str(')'))

#intermediate_nodes

In [ ]:
# Create csv of the coordinates for the intermediate nodes

intermediate_nodes['SectionID'] = intermediate_nodes['SectionID'].astype(str) + '(' + intermediate_nodes['SeqNumber'].astype(str) +')'
intermediate_nodes

intermediate_nodes.to_csv(r'../preprocesses/section_data/intermediate_coords_while.csv', header=True, index=True, mode='a')